In [17]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pickle

path_to_py = os.path.abspath('../src')



In [18]:
def clean_loan_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean and preprocess loan application data based on established cleaning steps.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Raw input DataFrame containing loan application data
    
    Returns:
    --------
    pd.DataFrame
        Cleaned DataFrame ready for model training/prediction
    """
    # Create a copy to avoid modifying the original dataframe
    data = df.copy()
    
    # 1. Fix Categorical Columns
    if 'term' in data.columns:
        data['term'] = data['term'].astype('category')
    
    if 'home_ownership' in data.columns:
        data['home_ownership'] = data['home_ownership'].fillna('OTHER')
        data['home_ownership'] = data['home_ownership'].astype('category')
    
    if 'purpose' in data.columns:
        data['purpose'] = data['purpose'].fillna(data['purpose'].mode()[0])
        data['purpose'] = data['purpose'].astype('category')
    
    # 2. Fix Percentage Columns
    if 'int_rate' in data.columns:
        data['int_rate'] = data['int_rate'].str.rstrip('%').astype('float')
    
    if 'revol_util' in data.columns:
        data['revol_util'] = data['revol_util'].str.rstrip('%').astype('float')
    
    # 3. Fix Employment Length
    if 'emp_length' in data.columns:
        def clean_emp_length(value):
            if pd.isna(value):
                return np.nan
            value = str(value).lower().strip()
            if '10+' in value:
                return 10
            elif '< 1' in value:
                return 0
            return int(value.split()[0])
        
        data['emp_length'] = data['emp_length'].apply(clean_emp_length)
    
    # 4. Drop Unnecessary Columns
    columns_to_drop = ['id', 'member_id', 'application_approved_flag', 'desc']
    data = data.drop(columns=[col for col in columns_to_drop if col in data.columns])
    
    # 5. Handle Credit Features
    data['no_credit_card_history'] = (
        data['total_bc_limit'].isnull() & 
        data['tot_hi_cred_lim'].isnull() & 
        data['bc_util'].isnull() & 
        data['percent_bc_gt_75'].isnull()
    ).astype(int)
    
    # Impute credit limit features
    for col in ['total_bc_limit', 'tot_hi_cred_lim']:
        if col in data.columns:
            median_val = data[data[col].notnull()][col].median()
            data[col] = data[col].fillna(data.apply(
                lambda x: 0 if x['no_credit_card_history'] == 1 else median_val, 
                axis=1
            ))
    
    # Impute utilization features
    for col in ['bc_util', 'percent_bc_gt_75']:
        if col in data.columns:
            mean_val = data[
                (data[col].notnull()) & 
                (data['total_bc_limit'] > 0)
            ][col].mean()
            data[col] = data[col].fillna(data.apply(
                lambda x: 0 if x['no_credit_card_history'] == 1 else mean_val,
                axis=1
            ))
    
    # 6. Handle Derogatory Feature
    if 'mths_since_last_major_derog' in data.columns:
        data['no_derog_history'] = data['mths_since_last_major_derog'].isnull().astype(int)
        data['mths_since_last_major_derog'] = data['mths_since_last_major_derog'].fillna(-1)
    
    # 7. Handle Inquiry Feature
    if 'mths_since_recent_inq' in data.columns:
        zero_inq_median = data[
            (data['inq_last_6mths'] == 0) & 
            (data['mths_since_recent_inq'].notnull())
        ]['mths_since_recent_inq'].median()
        
        medians_by_inq = data[
            (data['inq_last_6mths'] > 0) & 
            (data['mths_since_recent_inq'].notnull())
        ].groupby('inq_last_6mths')['mths_since_recent_inq'].median()
        
        data['mths_since_recent_inq'] = data.apply(
            lambda x: zero_inq_median if (pd.isnull(x['mths_since_recent_inq']) and x['inq_last_6mths'] == 0)
            else medians_by_inq[x['inq_last_6mths']] if pd.isnull(x['mths_since_recent_inq'])
            else x['mths_since_recent_inq'],
            axis=1
        )
    
    # 8. Handle Total Current Balance
    if 'tot_cur_bal' in data.columns:
        data.loc[data['no_credit_card_history'] == 1, 'tot_cur_bal'] = 0
        
        medians_by_purpose = data[
            (data['no_credit_card_history'] == 0) & 
            (data['tot_cur_bal'].notnull())
        ].groupby('purpose')['tot_cur_bal'].median()
        
        for purpose in data['purpose'].unique():
            mask = (
                (data['tot_cur_bal'].isnull()) & 
                (data['purpose'] == purpose) & 
                (data['no_credit_card_history'] == 0)
            )
            data.loc[mask, 'tot_cur_bal'] = medians_by_purpose[purpose]
    
    # 9. Handle Employment Length Missing Values
    if 'emp_length' in data.columns:
        data['emp_length_missing'] = data['emp_length'].isnull().astype(int)
        medians_by_ownership = data.groupby('home_ownership')['emp_length'].median()
        
        for ownership in data['home_ownership'].unique():
            mask = (data['emp_length'].isnull()) & (data['home_ownership'] == ownership)
            data.loc[mask, 'emp_length'] = medians_by_ownership[ownership]
    
    # 10. Handle Revolving Utilization
    if 'revol_util' in data.columns:
        median_val = data['revol_util'].median()
        data['revol_util'] = data['revol_util'].fillna(median_val)
    
    return data

In [19]:
def prepare_data_for_inference(new_df, preprocessing_params):
    """Prepare new data using saved preprocessing parameters"""
    df = new_df.copy()

    # 1. Categorical Encodings
    df['home_ownership_encoded'] = df['home_ownership'].map(preprocessing_params['home_ownership_risk'])
    # Purpose encoding
    def encode_purpose(purpose):
        purpose_groups = preprocessing_params['purpose_groups']
        if purpose in purpose_groups['low_risk']:
            return 0
        elif purpose in purpose_groups['medium_risk']:
            return 1
        else:
            return 2
    df['purpose_encoded'] = df['purpose'].apply(encode_purpose)
    df['term_encoded'] = df['term'].apply(lambda x: 0 if '36' in x else 1)

    # Drop original categorical columns
    df = df.drop(['home_ownership', 'purpose', 'term'], axis=1)

    # 2. Log transformations
    df['log_annual_inc'] = np.log1p(df['annual_inc'])
    df['log_loan_amt'] = np.log1p(df['loan_amnt'])
    df['log_dti'] = np.log1p(df['dti'])

    # 3. Standard scaling
    for feature in preprocessing_params['scale_features']:
        df[f'{feature}_scaled'] = preprocessing_params['scaler_dict'][feature].transform(df[[feature]])

    # 4. Special handling for derogatory marks
    mask = df['mths_since_last_major_derog'] >= 0
    df['mths_since_derog_scaled'] = df['mths_since_last_major_derog'].copy()
    df.loc[mask, 'mths_since_derog_scaled'] = preprocessing_params['derog_scaler'].transform(
        df.loc[mask, ['mths_since_last_major_derog']])

    # 5. Feature Engineering
    df['income_to_loan_ratio'] = df['log_annual_inc'] / df['log_loan_amt']
    df['int_rate_dti'] = df['int_rate_scaled'] * df['log_dti']
    df['risk_score'] = (df['int_rate_scaled'] + 
                       df['log_dti'] + 
                       df['revol_util_scaled'] - 
                       df['income_to_loan_ratio'])

    # 6. Drop original features
    df = df.drop(preprocessing_params['columns_to_drop'], axis=1)

    # 7. Ensure correct column order
    df = df[preprocessing_params['feature_names']]

    return df

In [29]:
def load_model_and_predict(data, model_path, preprocessing_path):
    """Load saved model and make predictions"""
    # Define model architecture
    class LoanDefaultModel(nn.Module):
        def __init__(self, input_dim, hidden_dims=[128, 64, 32]):
            super(LoanDefaultModel, self).__init__()
            
            self.input_bn = nn.BatchNorm1d(input_dim)
            
            layers = []
            prev_dim = input_dim
            
            for hidden_dim in hidden_dims:
                layers.extend([
                    nn.Linear(prev_dim, hidden_dim),
                    nn.BatchNorm1d(hidden_dim),
                    nn.ReLU(),
                    nn.Dropout(0.3)
                ])
                prev_dim = hidden_dim
            
            self.hidden_layers = nn.Sequential(*layers)
            self.output_layer = nn.Linear(hidden_dims[-1], 1)
        
        def forward(self, x):
            x = self.input_bn(x)
            x = self.hidden_layers(x)
            return torch.sigmoid(self.output_layer(x))

    # Load preprocessing parameters
    with open(preprocessing_path, 'rb') as f:
        preprocessing_params = pickle.load(f)
    
    # Prepare data
    processed_data = prepare_data_for_inference(data, preprocessing_params)

    processed_data = processed_data.drop('bad_flag', axis=1)
    
    # Load model
    checkpoint = torch.load(model_path)
    model = LoanDefaultModel(
        input_dim=checkpoint['input_dim'],
        hidden_dims=checkpoint['hidden_dims']
    )
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    # Convert to tensor
    X = torch.FloatTensor(processed_data.values)
    
    # Make predictions
    with torch.no_grad():
        predictions = model(X).numpy()
    
    return predictions

In [30]:
# Example usage on new data:
new_data = pd.read_csv(os.path.join(path_to_py, "data", "testing_loan_data.csv"))

display(new_data.head())

clean_data = clean_loan_data(new_data)

/var/folders/q0/7qtgvcgn1sxg_2jgqvvm73wr0000gn/T/ipykernel_33061/461941989.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  new_data = pd.read_csv(os.path.join(path_to_py, "data", "testing_loan_data.csv"))


,id,member_id,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,desc,purpose,...,inq_last_6mths,mths_since_recent_inq,revol_util,total_bc_limit,mths_since_last_major_derog,tot_hi_cred_lim,tot_cur_bal,application_approved_flag,internal_score,bad_flag
0,20000001,22419852,10000,36 months,22.15%,8 years,RENT,37000.0,NaN,debt_consolidation,...,1,3.0,73.10%,16200,NaN,14877.170280,36809,1,131,NaN
1,20000002,22349118,1400,36 months,18.24%,6 years,RENT,41000.0,NaN,other,...,0,9.0,11.50%,4000,NaN,4097.304770,19536,1,19,NaN
2,20000003,22398818,7000,36 months,12.49%,3 years,RENT,68900.0,NaN,debt_consolidation,...,0,11.0,48.10%,11900,80.0,12688.495160,241465,1,92,NaN
3,20000004,22419015,18000,60 months,16.29%,9 years,MORTGAGE,41000.0,NaN,debt_consolidation,...,1,0.0,38.10%,7600,73.0,7908.799817,179757,1,235,NaN
4,20000005,22388614,12000,36 months,12.99%,10+ years,MORTGAGE,64000.0,NaN,home_improvement,...,0,NaN,57.90%,21000,NaN,19378.561060,31953,1,157,NaN


In [31]:
display(clean_data.head())
clean_data.info()

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,percent_bc_gt_75,bc_util,dti,...,revol_util,total_bc_limit,mths_since_last_major_derog,tot_hi_cred_lim,tot_cur_bal,internal_score,bad_flag,no_credit_card_history,no_derog_history,emp_length_missing
0,10000,36 months,22.15,8.0,RENT,37000.0,debt_consolidation,80.0,83.0,28.51,...,73.1,16200,-1.0,14877.170280,36809.0,131,NaN,0,1,0
1,1400,36 months,18.24,6.0,RENT,41000.0,other,0.0,0.0,26.58,...,11.5,4000,-1.0,4097.304770,19536.0,19,NaN,0,1,0
2,7000,36 months,12.49,3.0,RENT,68900.0,debt_consolidation,60.0,75.9,6.60,...,48.1,11900,80.0,12688.495160,241465.0,92,NaN,0,0,0
3,18000,60 months,16.29,9.0,MORTGAGE,41000.0,debt_consolidation,33.3,61.1,20.61,...,38.1,7600,73.0,7908.799817,179757.0,235,NaN,0,0,0
4,12000,36 months,12.99,10.0,MORTGAGE,64000.0,home_improvement,75.0,67.0,24.61,...,57.9,21000,-1.0,19378.561060,31953.0,157,NaN,0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102505 entries, 0 to 102504
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   loan_amnt                    102505 non-null  int64   
 1   term                         102505 non-null  category
 2   int_rate                     102505 non-null  float64 
 3   emp_length                   102505 non-null  float64 
 4   home_ownership               102505 non-null  category
 5   annual_inc                   102505 non-null  float64 
 6   purpose                      102505 non-null  category
 7   percent_bc_gt_75             102505 non-null  float64 
 8   bc_util                      102505 non-null  float64 
 9   dti                          102505 non-null  float64 
 10  inq_last_6mths               102505 non-null  int64   
 11  mths_since_recent_inq        102505 non-null  float64 
 12  revol_util                   102505 non-null

In [39]:
predictions = load_model_and_predict(
    clean_data, 
    os.path.join(path_to_py, "models", "loan_default_model.pth"),
    os.path.join(path_to_py, "models", "processing_params.pkl")
)

clean_data = clean_data.drop('bad_flag', axis=1)
# Create DataFrame with predictions
# clean_data['bad_flag'] = predictions.squeeze()
# If you want binary predictions (0 or 1) instead of probabilities
clean_data['bad_flag'] = (predictions.squeeze() >= 0.5).astype(int)
clean_data['bad_flag'].value_counts()

/var/folders/q0/7qtgvcgn1sxg_2jgqvvm73wr0000gn/T/ipykernel_33061/1643607871.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path)


bad_flag
0    74783
1    27722
Name: count, dtype: int64

In [40]:
# Verify predictions
print("\nPrediction Summary:")
print(clean_data['bad_flag'].describe())

print("Binary predictions distribution:")
print(clean_data['bad_flag'].value_counts(normalize=True))


Prediction Summary:
count    102505.000000
mean          0.270445
std           0.444192
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: bad_flag, dtype: float64
Binary predictions distribution:
bad_flag
0    0.729555
1    0.270445
Name: proportion, dtype: float64


In [41]:
clean_data.to_csv(os.path.join(path_to_py, "data", "test_result.csv"), index=False)